In [32]:
import datetime
import pathlib

from nltk import word_tokenize
etds = pathlib.Path("../tmp/etds")
etd_paths = [etd for etd in etds.iterdir()]

In [43]:
len(etd_paths)

6287

In [44]:
etd_paths[34]

PosixPath('../tmp/etds/wg700tb0779.txt')

In [12]:
etd_paths[56].stat().st_size

637

In [14]:
empty_etds = []
for i,etd in enumerate(etd_paths):
    if etd.stat().st_size < 1:
        print(f"{i} {etd.name} size is less than one")
        empty_etds.append(etd)
    

80 my460fq1046.txt size is less than one
140 yd796zm6740.txt size is less than one
337 yq071tj0740.txt size is less than one
450 yx656fs6181.txt size is less than one
781 ns144db9649.txt size is less than one
820 kr698ww0913.txt size is less than one
1216 sy486tp5223.txt size is less than one
1219 dh727rv8642.txt size is less than one
1411 bd697wr9786.txt size is less than one
1505 tk151jc7099.txt size is less than one
1731 tm772zz7352.txt size is less than one
1940 hv283bs7821.txt size is less than one
1992 xx259tf5123.txt size is less than one
2001 yb440sg1391.txt size is less than one
2027 gk306wd4742.txt size is less than one
2260 zx064tb7256.txt size is less than one
2331 hz462yv9251.txt size is less than one
2479 fq899hc0553.txt size is less than one
2553 sn866kq8471.txt size is less than one
2678 mk114rp1100.txt size is less than one
2786 hk239ky7693.txt size is less than one
3136 pk521wd7589.txt size is less than one
3313 dr308cc5354.txt size is less than one
3391 sk624fg3275.t

In [15]:
len(empty_etds)

45

In [33]:
raw_text = etd_paths[6].read_text()

In [34]:
raw_text.find("Abstract", 2161)

10316

In [22]:
etd_paths[6].name

'kh271wr8164.txt'

In [35]:
kh271wr8164_tokens = word_tokenize(raw_text)

In [38]:
kh271wr8164_tokens[0:2]

['ANALYSIS', 'OF']

In [ ]:
print(f"""Counts:
\tAbstract {kh271wr8164_tokens.count("Abstract")}
\tReferences {kh271wr8164_tokens.count("References")} """)

In [48]:
import requests
import lxml.etree as etree
ns = {"mods": "http://www.loc.gov/mods/v3"}

In [59]:
for i,thesis in enumerate(etd_paths):
    abstract_file = pathlib.Path(f"/Users/jpnelson/2020/sul-dlss/tmp/etds/abstracts/{thesis.name}")
    if abstract_file.exists():
        continue
    purl_url = f"https://purl.stanford.edu/{thesis.name[:-4]}.xml"
    purl_result = requests.get(purl_url)
    if not i%25:
        print(".", end="")
    if not i%100:
        print(f"{i}", end="")
    try:
        purl_xml = etree.XML(purl_result.text.encode())
    except etree.XMLSyntaxError:
        print(f"\nXML error for {i} {thesis.name}")
    abstract = purl_xml.find("mods:mods/mods:abstract", namespaces=ns)
    if abstract is None:
        print(f"\nNo abstract {i} {thesis.name}")
        continue
    with abstract_file.open("w+") as file_obj:
        file_obj.write(abstract.text)
    


No abstract 2170 dg421jb1685.txt

No abstract 2974 dz885gy0747.txt

No abstract 3710 world-accessible-etd-full-text.tar.gz


In [17]:
import lxml.etree as etree
import requests

ns = {"mods": "http://www.loc.gov/mods/v3"}

def get_department(druid: str):
    purl_url = f"https://purl.stanford.edu/{druid}.xml"
    try:
        purl_result = requests.get(purl_url)
    except:
        print(f"Request error for {druid} with {purl_url}")
        return ''
    try:
        purl_xml = etree.XML(purl_result.text.encode())
    except etree.XMLSyntaxError:
        print(f"\nXML Syntax Error for {druid}")
        return ''
    corporate_names = purl_xml.findall("mods:mods/mods:name[@type='corporate']/mods:namePart", namespaces=ns)
    department = []
    for name in corporate_names:
        if name.text.startswith("Stanford University"):
            continue
        department.append(name.text)
    return ','.join(department)

In [18]:
import datetime
druid_departments = {}
start = datetime.datetime.utcnow()
print(start)
for i,etd in enumerate(etd_paths):
    if etd.is_dir():
        continue
    druid = etd.name[:-4]
    departments = get_department(druid)
    druid_departments[druid] = departments
    if not i%25:
        print(".", end="")
    if not i%250:
        print(f"{i}", end="")
end = datetime.datetime.utcnow()
print(f"Finished at {end} total time {(end-start).seconds / 60.} minutes")

2020-10-20 16:22:50.518016
.0..........250..........500..........750..........1000..........1250..........1500.......Request error for sf979mz6738 with https://purl.stanford.edu/sf979mz6738.xml
...1750...
XML Syntax Error for tt337yq1092
.......2000.......
XML Syntax Error for lm_c
...2250
XML Syntax Error for gs372pn4215
..........2500..........2750.........
XML Syntax Error for lm_train
.3000..........3250..........3500
XML Syntax Error for world-accessible-etd-full-
.
XML Syntax Error for abstr
.........3750..........4000..........4250..........4500..........4750....
XML Syntax Error for lm_val
......5000..........5250..........5500..........5750..........6000..........6250.Finished at 2020-10-20 17:44:09.836560 total time 81.31666666666666 minutes


In [20]:
len(druid_departments)

6288

In [25]:
for row in list(druid_departments.keys())[0:4]:
    print(druid_departments[row])

Department of Computer Science.
Graduate School of Education.
Department of Geological and Environmental Sciences.
Department of Bioengineering.


In [26]:
import pandas as pd
abstract_df = pd.read_pickle("./data/abstracts.pkl")

In [27]:
def add_department(druid: str) -> str:
    return druid_departments.get(druid)

abstract_df['departments'] = abstract_df.druids.apply(add_department)

In [28]:
abstract_df.head()

,druids,abstracts,abstracts_cleaned,departments
0,yq942qc7340,Metaverses are virtual worlds where users crea...,metaverses virtual worlds users create entire ...,Department of Computer Science.
1,yx753dx0216,"In this dissertation, I use two novel test sco...",dissertation use two novel test score data se...,Graduate School of Education.
2,yd874rr2274,A 20-minute documentary film was created to ac...,minute documentary film created accelerate ...,Department of Geological and Environmental Sci...
3,sh260yn9550,The usage of hydrogen as an alternative energy...,usage hydrogen alternative energy carrier beco...,Department of Bioengineering.
4,nk877ng0918,High-order methods in Computational Fluid Dyna...,high order methods computational fluid dynamic...,Department of Aeronautics and Astronautics.


In [30]:
abstract_df.to_pickle("./data/abstracts.pkl")

In [39]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
etd_encoded = le.fit_transform(kh271wr8164_tokens)

In [40]:
etd_encoded[0:5]

array([ 697, 2880, 1714, 3752, 1606])

In [41]:
len(etd_encoded)

38798